In [31]:
import plotly.graph_objs as go
from ipywidgets import widgets
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
import numpy as np
import plotly.plotly as py
from skimage import io
import nibabel as nib
import SimpleITK as sitk
from plotly.grid_objs import Grid, Column
import time
import pandas as pd
from plotly import tools
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

N = 100
x = np.random.randn(N)+1
y_orig = np.random.randn(N)+2
y_new = y_orig

global saved_points 
saved_points = {'idxs': [], 'x': [], 'y': []}

# Assigns plot array the original values
def resetValues(b):
    f.data[0].x = x
    f.data[0].y = y_orig
    scatter = f.data[0]
    colors = ['#a3a7e4'] * 100
    scatter.marker.color = colors
    scatter.marker.size = [10] * 100
    f.layout.hovermode = 'closest'

# Generates button to reset plot
reset_bt = widgets.Button(description="Reset")

# Box to specify what value to give the selected points
text_input = widgets.Text(
    description='y:',
    value='0',
)

# Callback for reset button
reset_bt.on_click(resetValues)

# Generates box and button
container = widgets.HBox(children=[text_input, reset_bt])
display(container)

# Generates figure
f = go.FigureWidget([go.Scatter(x=x, y=y_orig, mode = 'markers')])

# Specify figure data
scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'

# create our callback function to update points (save points)
def update_point(trace, points, selector):
    y_new[points.point_inds] = float(text_input.value)
    print(points)
    f.data[0].x = x
    f.data[0].y = y_new
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#f23535'
        s[i] = 20
        scatter.marker.color = c
        scatter.marker.size = s
        #print(c)
    #print(y_orig)
    saved_points['idxs'].append(points.point_inds[0])
    saved_points['x'].append(points.xs[0])
    saved_points['y'].append(points.ys[0])

# Update points 
scatter.on_click(update_point)
scatter.on_selection(update_point)

# Show plot
f

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
               …

In [49]:
def generate_map(orig_vol, idxs, bold):
    
    selected_vol = orig_vol[:,:,:,idxs] # only save selected timepoints to work with a smaller volume
    selected_vol[bold[idxs] < 0] = -selected_vol[:,:,:,idxs] # multiply by minus 1 volume of selected timepoint that had a negative BOLD amplitude
    cap_map = np.mean(selected_vol, axis=-1) # Generate the map by calculating the mean
    
    return(cap_map)
    

AttributeError: 'Scatter' object has no attribute 'callba'

In [23]:
nifti_image_filename = 'MNI152_T1_3mm_brain.nii.gz'
tif_image_filename = 'template.tif'

reader = sitk.ImageFileReader()
reader.SetImageIO('NiftiImageIO')
reader.SetFileName(nifti_image_filename)
image = reader.Execute()
writer = sitk.ImageFileWriter()
writer.SetFileName(tif_image_filename)
writer.Execute(image)

In [42]:
vol = io.imread('attention-mri.tif')
volume = vol.T
r, c = volume[0].shape

my_columns = []
nr_frames = 68
#grid = pd.DataFrame()

for k in range(nr_frames):
    #grid = grid.append({'z{}'.format(k + 1): (6.7 - k * 0.1) * np.ones((r, c)), 'surfc{}'.format(k + 1): np.flipud(volume[67 - k])}, ignore_index=True)
    my_columns.extend(
        [Column((6.7 - k * 0.1) * np.ones((r, c)), 'z{}'.format(k + 1)),
         Column(np.flipud(volume[67 - k]), 'surfc{}'.format(k + 1))]
    )
grid = Grid(my_columns)


In [43]:
pl_bone = [
    [0.0, 'rgb(0, 0, 0)'],
    [0.05, 'rgb(10, 10, 14)'],
    [0.1, 'rgb(21, 21, 30)'],
    [0.15, 'rgb(33, 33, 46)'],
    [0.2, 'rgb(44, 44, 62)'],
    [0.25, 'rgb(56, 55, 77)'],
    [0.3, 'rgb(66, 66, 92)'],
    [0.35, 'rgb(77, 77, 108)'],
    [0.4, 'rgb(89, 92, 121)'],
    [0.45, 'rgb(100, 107, 132)'],
    [0.5, 'rgb(112, 123, 143)'],
    [0.55, 'rgb(122, 137, 154)'],
    [0.6, 'rgb(133, 153, 165)'],
    [0.65, 'rgb(145, 169, 177)'],
    [0.7, 'rgb(156, 184, 188)'],
    [0.75, 'rgb(168, 199, 199)'],
    [0.8, 'rgb(185, 210, 210)'],
    [0.85, 'rgb(203, 221, 221)'],
    [0.9, 'rgb(220, 233, 233)'],
    [0.95, 'rgb(238, 244, 244)'],
    [1.0, 'rgb(255, 255, 255)']
]

In [39]:
data=[
    dict(
        type='surface', 
        zsrc=grid.loc[:,'z1'].dropna(),
        surfacecolorsrc=grid.loc[:,'surfc1'].dropna(),
        colorscale=pl_bone,
        colorbar=dict(thickness=20, ticklen=4)
    )
]

frames=[]
for k in range(nr_frames):
    frames.append(
        dict(
            data=[dict(zsrc=grid.loc[:,'z{}'.format(k + 1)].dropna(),
                       surfacecolorsrc=grid.loc[:,'surfc{}'.format(k + 1)].dropna())],
            name='frame{}'.format(k + 1)
        )
    )

sliders=[
    dict(
        steps=[dict(method='animate',
                    args= [['frame{}'.format(k + 1)],
                            dict(mode='immediate',
                                 frame= dict(duration=70, redraw= False),
                                 transition=dict(duration=0))],
                    label='{:d}'.format(k+1)) for k in range(61)], 
        transition= dict(duration=0),
        x=0,
        y=0, 
        currentvalue=dict(font=dict(size=12), 
                          prefix='slice: ', 
                          visible=True, 
                          xanchor='center'
                         ),  
        len=1.0
    )
]

axis3d = dict(
    showbackground=True, 
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",      
    zerolinecolor="rgb(255, 255, 255)",  
)

layout3d = dict(
         title='Slices in volumetric data', 
         font=dict(family='Balto'),
         width=600,
         height=600,
         scene=dict(xaxis=(axis3d),
                    yaxis=(axis3d), 
                    zaxis=dict(axis3d, **dict(range=[-0.1, 6.8], autorange=False)), 
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus=[
             dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=1.3,
                  xanchor='right',
                  yanchor='top',
                  pad=dict(t=0, r=10),
                  buttons=[dict(label='Play',
                                method='animate',
                                args=[
                                    None, 
                                    dict(frame=dict(duration=70, redraw=False),
                                         transition=dict(duration=0),
                                         fromcurrent=True,
                                         mode='immediate')
                                ])])
         ],
        sliders=sliders
)

AttributeError: 'Grid' object has no attribute 'loc'

In [58]:
data=[
    dict(
        type='surface',
        zsrc=grid.get_column_reference('z1'),
        surfacecolorsrc=grid.get_column_reference('surfc1'),
        colorscale=pl_bone,
        colorbar=dict(thickness=20, ticklen=4)
    )
]

frames=[]
for k in range(nr_frames):
    frames.append(
        dict(
            data=[dict(zsrc=grid.get_column_reference('z{}'.format(k + 1)),
                       surfacecolorsrc=grid.get_column_reference('surfc{}'.format(k + 1)))],
            name='frame{}'.format(k + 1)
        )
    )

sliders=[
    dict(
        steps=[dict(method='animate',
                    args= [['frame{}'.format(k + 1)],
                            dict(mode='immediate',
                                 frame= dict(duration=70, redraw= False),
                                 transition=dict(duration=0))],
                    label='{:d}'.format(k+1)) for k in range(68)], 
        transition= dict(duration=0),
        x=0,
        y=0, 
        currentvalue=dict(font=dict(size=12), 
                          prefix='slice: ', 
                          visible=True, 
                          xanchor='center'
                         ),  
        len=1.0
    )
]

axis3d = dict(
    showbackground=True, 
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",      
    zerolinecolor="rgb(255, 255, 255)",  
)

layout3d = dict(
         title='Slices in volumetric data', 
         width=600,
         height=600,
         scene=dict(xaxis=(axis3d),
                    yaxis=(axis3d), 
                    zaxis=dict(axis3d, **dict(range=[-0.1, 6.8], autorange=False)), 
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus=[
             dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=1.3,
                  xanchor='right',
                  yanchor='top',
                  pad=dict(t=0, r=10),
                  buttons=[dict(label='Play',
                                method='animate',
                                args=[
                                    None, 
                                    dict(frame=dict(duration=70, redraw=False),
                                         transition=dict(duration=0),
                                         fromcurrent=True,
                                         mode='immediate')
                                ])])
         ],
        sliders=sliders
)

In [59]:
fig=dict(data=data, layout=layout3d, frames=frames)
#icreate_animations(fig, filename='animslicesHead'+str(time.time()))
iplot(fig, filename='animslicesHead'+str(time.time()))

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [{'type': 'Surface', 'zsrc': '', 'surfacecolorsrc': '', 'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.05, 'rgb(10, 10, 14)'], [0.1, 'rgb(21, 21, 30)'], [0.15, 'rgb(33, 33, 46)'], [0.2, 'rgb(44, 44, 62)'], [0.25, 'rgb(56, 55, 77)'], [0.3, 'rgb(66, 66, 92)'], [0.35, 'rgb(77, 77, 108)'], [0.4, 'rgb(89, 92, 121)'], [0.45, 'rgb(100, 107, 132)'], [0.5, 'rgb(112, 123, 143)'], [0.55, 'rgb(122, 137, 154)'], [0.6, 'rgb(133, 153, 165)'], [0.65, 'rgb(145, 169, 177)'], [0.7, 'rgb(156, 184, 188)'], [0.75, 'rgb(168, 199, 199)'], [0.8, 'rgb(185, 210, 210)'], [0.85, 'rgb(203, 221, 221)'], [0.9, 'rgb(220, 233, 233)'], [0.95, 'rgb(238, 244, 244)'], [1.0, 'rgb(255, 255, 255)']], 'colorbar': {'thickness': 20, 'ticklen': 4}}]

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A list o

In [27]:
df

,surfc1,z1,surfc2,z2,surfc3,z3,surfc4,z4,surfc5,z5,...,surfc64,z64,surfc65,z65,surfc66,z66,surfc67,z67,surfc68,z68
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6.6000000000000005, 6.6000000000000005, 6.60...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5,...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4,...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
grid.loc[:,'z1'].dropna()

0    [[6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7, 6.7,...
Name: z1, dtype: object

In [10]:
print(r)
print(c)

232
256


In [32]:
init_notebook_mode(connected=True)